In [2]:
import numpy as np
import numpy.linalg as al
import scipy.stats as st


def one_hot(y, fill_k=False, one_not=False):
    """Map to one-hot encoding."""
    # Check labels
    labels = np.unique(y)

    # Number of classes
    K = len(labels)

    # Number of samples
    N = y.shape[0]

    # Preallocate array
    if one_not:
        Y = -np.ones((N, K))
    else:
        Y = np.zeros((N, K))

    # Set k-th column to 1 for n-th sample
    for n in range(N):

        # Map current class to index label
        y_n = (y[n] == labels)

        if fill_k:
            Y[n, y_n] = y_n
        else:
            Y[n, y_n] = 1

    return Y, labels


def regularize_matrix(A, a=0.0):
    """
    Regularize matrix by ensuring minimum eigenvalues.
    INPUT   (1) array 'A': square matrix
            (2) float 'a': constraint on minimum eigenvalue
    OUTPUT  (1) array 'B': constrained matrix
    """
    # Check for square matrix
    N, M = A.shape
    if not N == M:
        raise ValueError('Matrix not square.')

    # Check for valid matrix entries
    if np.any(np.isnan(A)) or np.any(np.isinf(A)):
        raise ValueError('Matrix contains NaNs or infinities.')

    # Check for non-negative minimum eigenvalue
    if a < 0:
        raise ValueError('minimum eigenvalue cannot be negative.')

    elif a == 0:
        return A

    else:
        # Ensure symmetric matrix
        A = (A + A.T) / 2

        # Eigenvalue decomposition
        E, V = al.eig(A)

        # Regularization matrix
        aI = a * np.eye(N)

        # Subtract regularization
        E = np.diag(E) + aI

        # Cap negative eigenvalues at zero
        E = np.maximum(0, E)

        # Reconstruct matrix
        B = np.dot(np.dot(V, E), V.T)

        # Add back subtracted regularization
        return B + aI


def is_pos_def(X):
    """Check for positive definiteness."""
    return np.all(np.linalg.eigvals(X) > 0)


def nullspace(A, atol=1e-13, rtol=0):
    """
    Compute an approximate basis for the nullspace of A.
    INPUT   (1) array 'A': 1-D array with length k will be treated
                as a 2-D with shape (1, k).
            (2) float 'atol': the absolute tolerance for a zero singular value.
                Singular values smaller than `atol` are considered to be zero.
            (3) float 'rtol': relative tolerance. Singular values less than
                rtol*smax are considered to be zero, where smax is the largest
                singular value.
                If both `atol` and `rtol` are positive, the combined tolerance
                is the maximum of the two; tol = max(atol, rtol * smax)
                Singular values smaller than `tol` are considered to be zero.
    OUTPUT  (1) array 'B': if A is an array with shape (m, k), then B will be
                an array with shape (k, n), where n is the estimated dimension
                of the nullspace of A.  The columns of B are a basis for the
                nullspace; each element in np.dot(A, B) will be
                approximately zero.
    """
    # Expand A to a matrix
    A = np.atleast_2d(A)

    # Singular value decomposition
    u, s, vh = al.svd(A)

    # Set tolerance
    tol = max(atol, rtol * s[0])

    # Compute the number of non-zero entries
    nnz = (s >= tol).sum()

    # Conjugate and transpose to ensure real numbers
    ns = vh[nnz:].conj().T

    return ns

In [35]:
import numpy as np
import scipy.stats as st
from scipy.sparse.linalg import eigs
from scipy.spatial.distance import cdist
import sklearn as sk
from sklearn.svm import LinearSVR
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_val_predict
from os.path import basename


class TransferComponentClassifier(object):
    """
    Class of classifiers based on Transfer Component Analysis.
    Methods contain component analysis and general utilities.
    """

    def __init__(self,
                 loss_function='logistic',
                 l2_regularization=1.0,
                 mu=1.0,
                 num_components=1,
                 kernel_type='rbf',
                 bandwidth=1.0,
                 order=2.0):
        """
        Select a particular type of transfer component classifier.
        Parameters
        ----------
        loss_function : str
            loss function for weighted classifier, options: 'logistic',
            'quadratic', 'hinge' (def: 'logistic')
        l2 : float
            l2-regularization parameter value (def:0.01)
        mu : float
            trade-off parameter (def: 1.0)
        num_components : int
            number of transfer components to maintain (def: 1)
        kernel_type : str
            type of kernel to use, options: 'rbf' (def: 'rbf')
        bandwidth : float
            kernel bandwidth for transfer component analysis (def: 1.0)
        order : float
            order of polynomial for kernel (def: 2.0)
        Returns
        -------
        None
        Attributes
        ----------
        loss
            which loss function to use
        is_trained
            whether the classifier has been trained on data already
        """
        self.loss = loss_function
        self.l2 = l2_regularization
        self.mu = mu
        self.num_components = num_components

        self.kernel_type = kernel_type
        self.bandwidth = bandwidth
        self.order = order

        # Initialize untrained classifiers
        if self.loss in ('lr', 'logr', 'logistic'):

            if l2_regularization:

                # Logistic regression model with fixed regularization
                self.clf = LinearRegression()

            else:
                # Logistic regression model, cross-validated for regularization
                self.clf = LinearRegressionCV()

        elif self.loss in ('squared', 'qd', 'quadratic'):

            if l2_regularization:

                # Least-squares model with fixed regularization
                self.clf = Ridge()

            else:
                # Least-squares model, cross-validated for regularization
                self.clf = Ridge()

        elif self.loss in ('hinge', 'linsvm', 'linsvr'):

            # Linear support vector machine
            self.clf = LinearSVR()

        elif self.loss in ('rbfsvr', 'rbfsvm'):

            # Radial basis function support vector machine
            self.clf = SVR()
        else:
            # Other loss functions are not implemented
            raise NotImplementedError

        # Maintain source and transfer data for computing kernels
        self.XZ = ''

        # Maintain transfer components
        self.C = ''

        # Whether model has been trained
        self.is_trained = False

        # Dimensionality of training data
        self.train_data_dim = ''

    def kernel(self, X, Z, type='rbf', order=2, bandwidth=1.0):
        """
        Compute kernel for given data set.
        Parameters
        ----------
        X : array
            data set (N samples by D features)
        Z : array
            data set (M samples by D features)
        type : str
            type of kernel, options: 'linear', 'polynomial', 'rbf',
            'sigmoid' (def: 'linear')
        order : float
            degree for the polynomial kernel (def: 2.0)
        bandwidth : float
            kernel bandwidth (def: 1.0)
        Returns
        -------
        array
            kernel matrix (N+M by N+M)
        """
        # Data shapes
        N, DX = X.shape
        M, DZ = Z.shape

        # Assert equivalent dimensionalities
        if not DX == DZ:
            raise ValueError('Dimensionalities of X and Z should be equal.')

        # Select type of kernel to compute
        if type == 'linear':
            # Linear kernel is data outer product
            return np.dot(X, Z.T)
        elif type == 'polynomial':
            # Polynomial kernel is an exponentiated data outer product
            return (np.dot(X, Z.T) + 1)**p
        elif type == 'rbf':
            # Radial basis function kernel
            return np.exp(-cdist(X, Z) / (2.*bandwidth**2))
        elif type == 'sigmoid':
            # Sigmoidal kernel
            return 1./(1 + np.exp(np.dot(X, Z.T)))
        else:
            raise NotImplementedError('Loss not implemented yet.')

    def transfer_component_analysis(self, X, Z):
        """
        Transfer Component Analysis.
        Parameters
        ----------
        X : array
            source data set (N samples by D features)
        Z : array
            target data set (M samples by D features)
        Returns
        -------
        C : array
            transfer components (D features by num_components)
        K : array
            source and target data kernel distances
        """
        # Data shapes
        N, DX = X.shape
        M, DZ = Z.shape

        # Assert equivalent dimensionalities
        if not DX == DZ:
            raise ValueError('Dimensionalities of X and Z should be equal.')

        # Compute kernel matrix
        XZ = np.concatenate((X, Z), axis=0)
        K = self.kernel(XZ, XZ, type=self.kernel_type,
                        bandwidth=self.bandwidth)

        # Ensure positive-definiteness
        if not is_pos_def(K):
            print('Warning: covariate matrices not PSD.')

            regct = -6
            while not is_pos_def(K):
                print('Adding regularization: ' + str(10**regct))

                # Add regularization
                K += np.eye(N + M)*10.**regct

                # Increment regularization counter
                regct += 1

        # Normalization matrix
        L = np.vstack((np.hstack((np.ones((N, N))/N**2,
                                  -1*np.ones((N, M))/(N*M))),
                       np.hstack((-1*np.ones((M, N))/(N*M),
                                  np.ones((M, M))/M**2))))

        # Centering matrix
        H = np.eye(N + M) - np.ones((N + M, N + M)) / float(N + M)

        # Matrix Lagrangian objective function: (I + mu*K*L*K)^{-1}*K*H*K
        J = np.dot(np.linalg.inv(np.eye(N + M) +
                   self.mu*np.dot(np.dot(K, L), K)),
                   np.dot(np.dot(K, H), K))

        # Eigenvector decomposition as solution to trace minimization
        _, C = eigs(J, k=self.num_components)

        # Discard imaginary numbers (possible computation issue)
        return np.real(C), K

    def fit(self, X, y, Z):
        """
        Fit/train a classifier on data mapped onto transfer components.
        Parameters
        ----------
        X : array
            source data (N samples by D features)
        y : array
            source labels (N samples by 1)
        Z : array
            target data (M samples by D features)
        Returns
        -------
        None
        """
        # Data shapes
        N, DX = X.shape
        M, DZ = Z.shape

        # Assert equivalent dimensionalities
        if not DX == DZ:
            raise ValueError('Dimensionalities of X and Z should be equal.')

        # Assert correct number of components for given dataset
        if not self.num_components <= N + M - 1:
            raise ValueError('''Number of components must be smaller than or
                             equal to the source sample size plus target sample
                             size plus 1.''')

        # Maintain source and target data for later kernel computations
        self.XZ = np.concatenate((X, Z), axis=0)

        # Transfer component analysis
        self.C, K = self.transfer_component_analysis(X, Z)

        # Map source data onto transfer components
        X = np.dot(K[:N, :], self.C)

        # Train a weighted classifier
        if self.loss in ('lr', 'logr', 'logistic'):

            # Logistic regression model with sample weights
            self.clf.fit(X, y)

        elif self.loss in ('squared', 'qd', 'quadratic'):

            # Least-squares model with sample weights
            self.clf.fit(X, y)

        elif self.loss in ('hinge', 'linsvm', 'linsvr'):

            # Linear support vector machine with sample weights
            self.clf.fit(X, y)
        else:
            # Other loss functions are not implemented
            raise NotImplementedError('Loss not implemented yet.')

        # Mark classifier as trained
        self.is_trained = True

        # Store training data dimensionality
        self.train_data_dim = DX

    def predict(self, Z):
        """
        Make predictions on new dataset.
        Parameters
        ----------
        Z : array
            new data set (M samples by D features)
        Returns
        -------
        preds : array
            label predictions (M samples by 1)
        """
        # Data shape
        M, D = Z.shape

        # If classifier is trained, check for same dimensionality
        if self.is_trained:
            if not self.train_data_dim == D:
                raise ValueError('''Test data is of different dimensionality
                                 than training data.''')

        # Compute kernel for new data
        K = self.kernel(Z, self.XZ, type=self.kernel_type,
                        bandwidth=self.bandwidth, order=self.order)

        # Map new data onto transfer components
        Z = np.dot(K, self.C)

        # Call scikit's predict function
        preds = self.clf.predict(Z)

        # For quadratic loss function, correct predictions
        if self.loss == 'quadratic':
            preds = (np.sign(preds)+1)/2.

        # Return predictions array
        return preds

    def get_params(self):
        """Get classifier parameters."""
        return self.clf.get_params()

    def is_trained(self):
        """Check whether classifier is trained."""
        return self.is_trained

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Split according to distribution

df = pd.read_csv("data\ICS\Concrete_Data_2.csv", ";")
df= df.fillna(0)

# Calculate Correlation between coloumns 
corr_values = []
highest_corr = 0
highest_col = 0

for col in df:
    corr = df["9.00"].corr(df[col])
    corr_values.append(corr)
    if corr >= max(corr_values) and corr < 1.0:
        highest_corr = corr
        highest_col = col
    #print("Correlation between the target and "+ str(col) + " : " + str(corr))
    #print(highest_corr)

# Selecto Corr >= 0,4 and sort data accordingly 
print ("Column: "+str(highest_col)+ " has the highest correlation with the target: " + str(highest_corr))
df = df.sort_values(by=[highest_col])
df = df.reset_index(drop=True)

# Split df in 3 equal parts
split = int(len(df[:1000])/3)
df_1 = df.loc[0:split,:]
df_2 = df.loc[split:split*2,:]
df_3 = df.loc[split*2:split*3,:]
print("Df has been split into 3 equal parts: ",df_1.shape,df_2.shape,df_3.shape)

df_src = df_2 #.append(df_2)
#print(df_src.head(), df_src.shape)

df_tar = df_3
#print(df_tar.head(), df_tar.shape)

# source
Xs = df_src.iloc[:,:-1]
Ys = df_src.iloc[:,-1]

# target_train
Xt = df_tar.iloc[:,:-1]
Yt = df_tar.iloc[:,-1]

Column: 1.00 has the highest correlation with the target: 0.49783558206163886
Df has been split into 3 equal parts:  (334, 9) (334, 9) (334, 9)


In [21]:
clf = TransferComponentClassifier()
assert type(clf) == TransferComponentClassifier
assert not clf.is_trained

X = rnd.randn(10, 2)
y = np.hstack((-np.ones((5,)), np.ones((5,))))
Z = rnd.randn(10, 2) + 1
clf = TransferComponentClassifier()
clf.fit(X, y, Z)
assert clf.is_trained

X = rnd.randn(10, 2)
y = np.hstack((-np.ones((5,)), np.ones((5,))))
Z = rnd.randn(10, 2) + 1
clf = TransferComponentClassifier()
clf.fit(X, y, Z)
u_pred = clf.predict(Z)
labels = np.unique(y)
assert len(np.setdiff1d(np.unique(u_pred), labels)) == 0

In [45]:
clf = TransferComponentClassifier()
assert type(clf) == TransferComponentClassifier
assert not clf.is_trained

clf.fit(Xs, Ys, Xt[:25])
u_pred = clf.predict(Xt[:25])


In [48]:
Yt[:25]

666    66.60
667    36.80
668    40.93
669    28.80
670    38.46
671    46.23
672    46.23
673    38.46
674    81.75
675    38.70
676    24.44
677    40.06
678    40.06
679    68.10
680    33.40
681    25.20
682    66.10
683    55.50
684    37.26
685    57.21
686    57.22
687    37.27
688    52.42
689    31.18
690    29.59
Name: 9.00, dtype: float64

In [49]:
u_pred

array([33.97998583, 33.91628892, 33.91869779, 33.87146743, 33.90598948,
       33.91227457, 33.91227457, 33.90598948, 33.86604595, 33.86604657,
       33.86604657, 33.89402225, 33.89402226, 33.86604595, 33.86950762,
       33.86950715, 33.86604597, 33.86604677, 33.89520816, 33.89953753,
       33.89953753, 33.89520816, 33.86604595, 33.86604595, 33.86604651])